In [ ]:
import os
import xml.etree.cElementTree as et
from tqdm import tqdm
from pathlib import Path
import pandas as pd

In [1]:
def xml_to_text(path):
    tree = et.parse(path)
    root = tree.getroot()
    text = []
    for i in root.iter("SECTION"):
        for j in i.iter("S"):
            text.append(j.text)
    return text
        

def fetch_data(path='dataset/top1000_complete', data_type="Documents_xml"):
    texts = []
    path = Path(path)
    directory_names = os.listdir('dataset/top1000_complete')
    for name in tqdm(directory_names): 
        new_path = path / name
        if data_type == "Documents_xml":
            final_path = new_path / "Documents_xml"/ Path(name + ".xml")
            texts.append(xml_to_text(final_path))
        else:
            final_path = new_path / 'summary' / Path(name + ".gold.txt")
            with open(final_path, 'r') as f:
                texts.append(f.read())
    
    return texts, directory_names


In [2]:
len(os.listdir('dataset/top1000_complete'))

1009

In [3]:
texts, directory_names = fetch_data()
summaries, d_name = fetch_data(data_type="summary")

#  create a dataframe storing text, summary and by taking intersection of d_name and directory_names
df = pd.DataFrame({'text': texts, 'summary': summaries, 'd_name': d_name, 'directory_names': directory_names})
df = df[df['d_name'] == df['directory_names']]
df = df.drop(['d_name', 'directory_names'], axis=1)
df.head()

100%|██████████| 1009/1009 [00:01<00:00, 867.63it/s]


,text,summary
0,[Human evaluations of machine translation (MT)...,Bleu: A Method For Automatic Evaluation Of Mac...
1,[A large number of current language processing...,TnT - A Statistical Part-Of-Speech Tagger\nTri...
2,[Current automatic summarizers usually rely on...,Sentence Reduction For Automatic Text Summariz...
3,[Even moderately long documents typically addr...,Advances In Domain Independent Linear Text Seg...
4,[Word sense disambiguation is often cast as a ...,A Simple Approach To Building Ensembles Of Nai...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1009 entries, 0 to 1008
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     1009 non-null   object
 1   summary  1009 non-null   object
dtypes: object(2)
memory usage: 23.6+ KB


In [5]:
df['text'] = df.apply(lambda x: ' '.join(x['text']), axis=1)
df['text'] = df.apply(lambda x: 'summarize: ' + x['text'], axis=1)
df.head()

,text,summary
0,summarize: Human evaluations of machine transl...,Bleu: A Method For Automatic Evaluation Of Mac...
1,summarize: A large number of current language ...,TnT - A Statistical Part-Of-Speech Tagger\nTri...
2,summarize: Current automatic summarizers usual...,Sentence Reduction For Automatic Text Summariz...
3,summarize: Even moderately long documents typi...,Advances In Domain Independent Linear Text Seg...
4,summarize: Word sense disambiguation is often ...,A Simple Approach To Building Ensembles Of Nai...


In [ ]:
df.to_csv("./main.csv", index=False)